In [ ]:
import serial
import numpy as np
import time
import keras.models as models
import socket

In [ ]:
serialCom = serial.Serial('COM6', 115200) # check your serial port
serialCom.setDTR(False)
time.sleep(1)
serialCom.flushInput()
serialCom.setDTR(True)

In [ ]:
model = models.load_model('model.h5')

In [ ]:
def fist(sample_length=256):
    X = []
    for i in range(sample_length):
        data = serialCom.readline()
        decoded_data = data.decode('utf-8').strip()
        X.append(decoded_data)
    X = np.array(X).reshape(1, sample_length)
    try:
        return model.predict(X.astype(float))
    except:
        return 0

In [ ]:
ESP_IP = '192.168.x.x' # Replace with the IP address of your ESP8266
PORT = 8888

def movement(state):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        s.connect((ESP_IP, PORT))      
        
        s.sendall((state + '\n').encode('utf-8'))
        print(f"Sent '{state}' fist state.")

        s.close()

    except Exception as e:
        print(f"Failed to send data: {e}")

def main():
    while True:

        if fist()>0.8:
            movement("forward")
        else:
            movement("stop")

if __name__ == "__main__":
    main()
